In [1]:
!pip install bs4
!pip install lxml

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 KB 3.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=abd44dacef0feee3e1a9c424292a4643747fac9391c9b6ceb09ece659df90fa0
  Stored in directory: /home/amr10211/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 17.9 MB/s eta 0:00:0000:0100:01m


In [2]:
from bs4 import BeautifulSoup
import os
from glob import glob
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
from dask import delayed
import pandas as pd
from distributed import Client
from dask_jobqueue import SLURMCluster
from IPython.display import display
import matplotlib.pyplot as plt
import time
import numpy as np
import pyarrow
from dask.diagnostics import ProgressBar
import time
import csv
from tqdm import tqdm
import io
import sys
from dask.diagnostics import ProgressBar
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
csv.field_size_limit(sys.maxsize)


# Set LOCAL to True for single-machine execution while developing
# Set LOCAL to False for cluster execution
LOCAL = False


if LOCAL:
    # This line creates a single-machine dask client
    client = Client()
else:    
    # This line creates a SLURM cluster dask and dask client
    # Logging outputs will be stored in /scratch/{your-netid}
    
    cluster = SLURMCluster(
                           # Memory and core limits should be sufficient here
                           memory='64GB', cores=8,

                            # Ensure that Dask uses the correct version of Python on the cluster
                            python='/scratch/work/public/dask/{}/bin/python'.format(dask.__version__),                           
                           
                            # Place the output logs in an accessible location
                            job_extra=['--export=NONE --output=/scratch/{}/slurm-%j.out'.format(os.environ['SLURM_JOB_USER'])]
    )

    cluster.submit_command = 'slurm'
    cluster.scale(200) 

    display(cluster)
    client = Client(cluster)

display(client)

/ext3/miniconda3/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35081 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.32.35.216:35081/status,
Dashboard: http://10.32.35.216:35081/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.32.35.216:41337,Workers: 0
Dashboard: http://10.32.35.216:35081/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [2]:
df = pd.read_csv('/vast/amr10211/opinions-data-sc.csv')

In [ ]:
# Function to extract text from HTML
def extract_text_from_html(html_string):
    try:
        if pd.isna(html_string) or not isinstance(html_string, str):
            return float('NaN')
        soup = BeautifulSoup(html_string, 'html.parser')
        return soup.get_text()
    except Exception as e:
        return str(e)

# Function to extract text from XML
def extract_text_from_xml(xml_string):
    try:
        if pd.isna(xml_string) or not isinstance(xml_string, str):
            return float('NaN')
        soup = BeautifulSoup(xml_string, 'lxml')
        return soup.get_text()
    except Exception as e:
        return str(e)
    
    
# Function to apply to each row
def get_decision_text(row):
    if pd.notnull(row['xml_harvard_text']):
        return row['xml_harvard_text']
    elif pd.notnull(row['html_with_citations_text']):
        return row['html_with_citations_text']
    elif pd.notnull(row['html_lawbox_text']):
        return row['html_lawbox_text']
    elif pd.notnull(row['html_text']):
        return row['html_text']
    elif pd.notnull(row['plain_text']):
        return row['plain_text']
    else:
        return None
    
    
# Iterate through DataFrame columns and apply the respective conversion function
df_text = df.copy()
df_text = df.iloc[:,0:19]
for col in tqdm(df.columns, desc="Processing Columns"):
    if 'html' in col:
        df_text[f'{col}_text'] = df[col].apply(extract_text_from_html)
    elif 'xml' in col:
        df_text[f'{col}_text'] = df[col].apply(extract_text_from_xml)
        
# Apply the function to each row of the DataFrame
df_text['decision_text'] = df_text.apply(get_decision_text, axis=1)
columns_to_drop = ['xml_harvard_text', 'html_with_citations_text', 'html_lawbox_text', 'html_text', 'plain_text', 'html_columbia_text','html_anon_2020_text']
df_text.drop(columns=columns_to_drop, inplace=True)

df_text.to_csv('/vast/amr10211/text-opinions-data-sc.csv')


Processing Columns:  78%|███████▊  | 21/27 [05:03<01:39, 16.52s/it]/home/amr10211/.local/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
Processing Columns: 100%|██████████| 27/27 [10:42<00:00, 23.81s/it]


In [125]:
df_text.head(1)

,resource_uri,id,absolute_url,cluster_id,cluster,author_id,author,joined_by,date_created,date_modified,author_str,per_curiam,joined_by_str,type,sha1,page_count,download_url,local_path,decision_text
0,NaN,90813,NaN,90813,NaN,3338.0,NaN,NaN,2010-04-28 16:02:43+00,2023-08-02 20:10:24.851392+00,Waite,f,NaN,010combined,c1f2f8751f0af48290e182d424e3fe12d31859b6,NaN,NaN,NaN,"\n\n \n *5\n \n Mr. Chief Justice Waite\n \n\n delivered the opinion of the court.\n \n\n This motion is denied. It does not appear from the motion papers that the decree appealed from is collectible under ordinary execution. The fair inference-from the statements in the papers is, that'the suit was instituted to subject the lands in dispute to the payment of a debt, and that no personal decree for money can be given against the appellants. The controversy seems to be as.to the rights of the appellee in the lands. The present bond is sufficient in amount to protect him against loss pending the suit from sales for taxes if he avails himself of the remedies by redemption and subrogation which the law affords. At any rate the circumstances of the parties do not appear to have changed in this particular since the security was taken originally.\n \n"
